In [1]:
import os
import pandas as pd
os.environ['CUDA_VISIBLE_DEVICES'] = '2' # nvidia-smi로 비어있는 gpu 확인하고 여기서 선택할것!

from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
)

from asset.tokenization_kobert import KoBertTokenizer
from asset import tokenization_kobert
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [2]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [3]:
model = EncoderDecoderModel.from_pretrained('./save_points/dacon-v0/checkpoint-30000')
model.eval()
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

In [4]:
def read_input(path):
    test_question = []
    test = pd.read_csv(path)
    for ut in test['질문']:
        test_question.append(ut)

    return test_question

In [5]:
test_q= read_input('./bigdata/test.csv')

In [6]:
def translate(text):
    embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    embeddings = {k: v for k, v in embeddings.items()}
    # generate 함수에 하이퍼파라미터 추가
    output_sequences = model.generate(
        **embeddings,
        max_length=65,
        temperature=0.9,
        top_k=1,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )
    
    tmp = trg_tokenizer.decode(output_sequences[0, 1:-1].cpu())
    return tmp

In [7]:
def translate_framework(text):
    pred = ""
    dividers = ["또한,","그리고"]
    for divider in dividers:
        
        if divider in text:
            text1 = text.split(divider)[0].strip()
            text2 = text.split(divider)[1].strip()
            pred = translate(text1) +" "+divider+" "+ translate(text2)
            return pred
    else:
        pred = translate(text)
        return pred
    

In [8]:
print("size of full : ",len(test_q))

size of full :  130


In [9]:
preds = []
for i in range(len(test_q)):
    sr_text = test_q[i] # dialect
    tg_text = translate_framework(sr_text) # translate
    preds.append(tg_text)
    if i<300:
        print("source_text : ",sr_text)
        print("result :      ",tg_text)
        print()
        print()

/home/leadawon5/gitfiles/venvs/myvenv/lib/python3.7/site-packages/transformers/generation/utils.py:1260: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"


source_text :  방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
result :       방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트 등이 있습니다. 이러한 페인트들은 각각의 특성과 용도에 맞게 선택하여 사용할 수 있습니다. 광명 또한, 원목사이딩의 단점은 주로 가격대가 높고 관리가 어렵다는 점입니다. 또한 습기에 약해 뒤틀림, 부서짐, 수축 및 팽창이 발생할 수 있다는 점도 단점으로 꼽힙니다. 또한 특히 곰팡이와 곤충에 노출될 경우 훼손될 가능성이 높다는 점도 감안해야 합니다.특히 습기가 많은 환경에서는 보다 신속하게


source_text :  도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
result :       도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 보통 높은 습도나 누수로 인해 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹


source_text :  큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
result :       큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더 들어갈 수 있다는 점도 단점으로 꼽힙니다. 이러한 점들을 감안하여 건축자 및 시공 업체들은 클라이언트에게 건축 재료의 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있는데, 이는 건강한 실내 환경을 유지하는 데 도움을 줍니다. 또한, 열전도율이 가장 낮아서 효율적인 단열 효과를 제공하며 시공이 간편하여 공사 기간을 단축하는 장점이 있습니다.  하지만 압출법단열판의 단점으로

In [10]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

(130, 512)

In [11]:
submit = pd.read_csv('./bigdata/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.039628,0.015293,-0.001330,0.000686,0.077017,0.021252,-0.016919,0.019856,-0.007895,...,0.006653,-0.044792,-0.014057,-0.027597,-0.022191,0.004917,0.037991,-0.009682,0.009912,0.037125
1,TEST_001,-0.034573,-0.016681,0.018237,-0.014298,0.091397,-0.009767,-0.019178,-0.004695,-0.008249,...,-0.067509,-0.001033,-0.001833,-0.025682,0.013643,0.050286,-0.013313,-0.023582,0.019273,0.002319
2,TEST_002,0.002209,-0.038676,-0.006808,0.015708,0.113381,-0.043061,0.002655,-0.029245,0.042291,...,-0.009037,-0.027377,0.059022,0.003549,-0.018805,0.010312,-0.000981,-0.020784,-0.019486,0.078727
3,TEST_003,-0.028849,0.000567,-0.001968,0.012996,0.095400,-0.058744,-0.069473,-0.006795,-0.030117,...,-0.039424,-0.010650,0.025045,-0.041679,-0.008780,0.061214,-0.013928,-0.000524,-0.063991,0.078699
4,TEST_004,0.006242,-0.021295,-0.012780,-0.010405,0.107459,-0.021475,0.051302,0.074226,0.020974,...,0.012778,-0.019018,0.039685,-0.008369,-0.031626,0.032361,-0.051368,0.000369,-0.011179,0.006354


In [12]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('./submit/dacon-v0_submit.csv', index=False)

In [13]:
sample="큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
print(sample.split("또한,"))

['큐블럭의 단점을 알려주세요. ', ' 압출법 단열판을 사용하는 것의 장점은 무엇인가요?']
